In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
%matplotlib inline 

In [ ]:
ames = pd.read_csv('./data/Ames_Housing_Price_Data.csv')
ames = ames.iloc[:,1:]

In [ ]:
ames.columns = ames.columns.str.lower()
ames.mssubclass = ames.mssubclass.astype('object')

## Missing Data Cleanning

Missingness: it seems that all the missing come from a non-existing feature, therefore I fill in all the numeric features with 0 and categorical features with ‘No [feature]’ or similar pattern. 

### Remarks:

*mansvnrtype / masvnrarea: missing because the house is solid brick? In that case, the price might be different.

*garage: row 433, 531 have garage type but with many missing values in garage specs

*‘Electrical’ : 1 missing data

*When 'Garagetype' is ‘no garage’, the value of 'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal', 'garagecond' will have multicollinearity since they will all be coded 0 or ‘No Garage’. For ML purposes, the overall model will include ‘garagetype’ or 'garagearea' only, if significant, the data set will be split in with/without garage for further analysis.

*The same with the basement features group.

In [ ]:
ames.loc[ames.lotfrontage.isnull(),'lotfrontage'] = 0
ames.loc[ames.alley.isnull(),'alley'] = 'No Alley'
ames.loc[ames.masvnrtype.isnull(),'masvnrtype'] = 'Missing'
ames.loc[ames.masvnrarea.isnull(),'masvnrarea'] = 0
ames.loc[ames.bsmtqual.isnull(),'bsmtqual'] = 'No Bsmt'
ames.loc[ames.bsmtcond.isnull(),'bsmtcond'] = 'No Bsmt'
ames.loc[ames.bsmtexposure.isnull(),'bsmtexposure'] = 'No Bsmt'
ames.loc[ames.bsmtfintype1.isnull(),'bsmtfintype1'] = 'No Bsmt'
ames.loc[ames.bsmtfinsf1.isnull(),'bsmtfinsf1'] = 0
ames.loc[ames.bsmtfintype2.isnull(),'bsmtfintype2'] = 'No Bsmt'
ames.loc[ames.bsmtfinsf2.isnull(),'bsmtfinsf2'] = 0
ames.loc[ames.bsmtunfsf.isnull(),'bsmtunfsf'] = 0
ames.loc[ames.totalbsmtsf.isnull(),'totalbsmtsf'] = 0
ames.loc[ames.bsmtfullbath.isnull(),'bsmtfullbath'] = 0
ames.loc[ames.bsmthalfbath.isnull(),'bsmthalfbath'] = 0
ames.loc[ames.fireplaces == 0,'fireplacequ'] = 'No Fireplace'
ames.loc[ames.garagetype.isnull(),'garagetype'] = 'No Garage'
ames.loc[ames.garagetype == 'No Garage','garageyrblt'] = 0
ames.loc[ames.garagetype == 'No Garage','garagefinish'] = 'No Garage'
ames.loc[ames.garagetype == 'No Garage','garagecars'] = 0
ames.loc[ames.garagetype == 'No Garage','garagearea'] = 0
ames.loc[ames.garagetype == 'No Garage','garagequal'] = 'No Garage'
ames.loc[ames.garagetype == 'No Garage','garagecond'] = 'No Garage'
ames.loc[ames.poolarea == 0,'poolqc'] = 'No Pool'
ames.loc[ames.fence.isnull(),'fence'] = 'No Fence'
ames.loc[ames.miscfeature.isnull(),'miscfeature'] = 'None'

In [ ]:
ames.loc[ames.garageyrblt.isnull(),]

In [ ]:
ames.loc[ames.electrical.isnull(),]

After the major data cleaning, 3 rows are still left with missing values. After checking the 3 records, the missing information cannot be imputed easily with existing information, so we decide to remove these 3 rows. 2577 rows of data will be included in the analysis.

In [ ]:
ames2 = ames.dropna(axis=0).copy()
ames2.shape

## Adding Features

- 'price_sqft' = 'saleprice' / 'grlivarea'
 
- 'house_age'  = 'yrsold' - 'yearbuilt'
 
- 'reno_age' = 'yrsold' - 'yearremodadd'
 
- bathroom # above ground: 'tbath_abvgr' = 'fullbath' + 'halfbath'*0.5

- bathroom # in basement: 'tbath_bsmt' = 'bsmtfullbath' + 'bsmthalfbath'*0.5
 
- bedroom/bathroom ratio (above ground): 'bbratio_abvgr' = 'bedroomabvgr' / 'tbath_abvgr'
  
There are two houses only have bathroom in basement, so the 'bbratio_abvgr' will be uncalculatble. 

To make sense of the date, impute the missing data with 10.

### Be aware of multicollinearity between the introduced variables

In [ ]:
ames2['price_sqft'] = ames2['saleprice'] / ames2['grlivarea']
ames2['house_age'] = ames2['yrsold'] - ames2['yearbuilt']
ames2['reno_age'] = ames2['yrsold'] - ames2['yearremodadd']
ames2.loc[ames2['reno_age']<0,'reno_age'] = 0
ames2['tbath_abvgr'] = ames2['fullbath'] + ames2['halfbath']*0.5
ames2['tbath_bsmt'] = ames2['bsmtfullbath'] + ames2['bsmthalfbath']*0.5
ames2['tbath'] = ames2['tbath_abvgr'] + ames2['tbath_bsmt']
ames2['bbratio_abvgr'] = ames['bedroomabvgr']/ames2['tbath_abvgr']
ames2.loc[ames2['bbratio_abvgr'].isnull(),'bbratio_abvgr'] = 10
ames2.loc[ames2['bbratio_abvgr']>10,'bbratio_abvgr'] = 10

## Preparing data set

### Categorical Variables

Creating a list of all categorial variable for future uses.

Check the Boxplots for all categorical variable against price per sqft

In [ ]:
factor_col = []
for col in ames2.columns:
    if ames2[col].dtype == 'O':
        factor_col.append(col)

In [ ]:
for col in factor_col:
    print(ames2[['price_sqft',col]].boxplot(by = col))

In [ ]:
for col in factor_col:
    print(ames2[col].value_counts())

By looking at the boxplots, it seems 'street', 'lotshape', 'loconfig', 'landslope', 'heatingqc', and 'fence' don't have strong influence on the price/sqft. May consider eliminating these features in the next steps.

Another observation is that many of the features have unbalanced values. This should be taken into consideration if any features are to be included in the model.

### Standardize Numeric Variables

Standardize all the numeric variables for better comparebility for the beta coefiicients.

In [ ]:
ames3 = ames2.copy()
for col in ames3.columns.tolist():
    if ames3[col].dtype != 'O':
        ames3[col] = (ames3[col] - ames3[col].mean()) / ames3[col].std()

In [ ]:
ames_numeric = ames2[ames2.columns[~ames2.columns.isin(factor_col)].tolist()]

### Dummify variables

In [ ]:
for col in factor_col:
    col_df = dummy_dict[col].drop(col+'_'+str(ames2[col].mode()[0]),axis=1)
    ames3 = pd.concat([ames3, col_df], join='inner',axis=1)
    ames3.drop(col, axis=1, inplace=True)
    
ames3.head()

### Create Features Series and Target DataFrame

In [ ]:
target = ames3['price_sqft']
full_features = ames3[['grlivarea','lotfrontage','lotarea','overallqual','overallcond','yearbuilt',
                       'yearremodadd','masvnrarea','bsmtfinsf1','bsmtfinsf2','bsmtunfsf','totalbsmtsf',
                       '1stflrsf','2ndflrsf','lowqualfinsf','bsmtfullbath','bsmthalfbath','fullbath',
                       'halfbath','bedroomabvgr','kitchenabvgr','totrmsabvgrd','fireplaces','garageyrblt',
                       'garagecars','garagearea','wooddecksf','openporchsf','enclosedporch','3ssnporch',
                       'screenporch','poolarea','miscval','mosold','yrsold','house_age',
                       'reno_age','tbath_abvgr','tbath_bsmt','tbath','bbratio_abvgr','mssubclass_30',
                       'mssubclass_40','mssubclass_45','mssubclass_50','mssubclass_60','mssubclass_70',
                       'mssubclass_75','mssubclass_80','mssubclass_85','mssubclass_90','mssubclass_120',
                       'mssubclass_150','mssubclass_160','mssubclass_180','mssubclass_190','mszoning_A (agr)',
                       'mszoning_C (all)','mszoning_FV','mszoning_I (all)','mszoning_RH','mszoning_RM',
                       'street_Grvl','alley_Grvl','alley_Pave','lotshape_IR1','lotshape_IR2','lotshape_IR3',
                       'landcontour_Bnk','landcontour_HLS','landcontour_Low','utilities_NoSewr',
                       'lotconfig_Corner','lotconfig_CulDSac','lotconfig_FR2','lotconfig_FR3','landslope_Mod',
                       'landslope_Sev','neighborhood_Blmngtn','neighborhood_Blueste','neighborhood_BrDale',
                       'neighborhood_BrkSide','neighborhood_ClearCr','neighborhood_CollgCr','neighborhood_Crawfor',
                       'neighborhood_Edwards','neighborhood_Gilbert','neighborhood_Greens','neighborhood_GrnHill',
                       'neighborhood_IDOTRR','neighborhood_Landmrk','neighborhood_MeadowV','neighborhood_Mitchel',
                       'neighborhood_NPkVill','neighborhood_NWAmes','neighborhood_NoRidge','neighborhood_NridgHt',
                       'neighborhood_OldTown','neighborhood_SWISU','neighborhood_Sawyer','neighborhood_SawyerW',
                       'neighborhood_Somerst','neighborhood_StoneBr','neighborhood_Timber','neighborhood_Veenker',
                       'condition1_Artery','condition1_Feedr','condition1_PosA','condition1_PosN','condition1_RRAe',
                       'condition1_RRAn','condition1_RRNe','condition1_RRNn','condition2_Artery','condition2_Feedr',
                       'condition2_PosA','condition2_PosN','condition2_RRAe','condition2_RRAn','condition2_RRNn',
                       'bldgtype_2fmCon','bldgtype_Duplex','bldgtype_Twnhs','bldgtype_TwnhsE','housestyle_1.5Fin',
                       'housestyle_1.5Unf','housestyle_2.5Fin','housestyle_2.5Unf','housestyle_2Story',
                       'housestyle_SFoyer','housestyle_SLvl','roofstyle_Flat','roofstyle_Gambrel','roofstyle_Hip',
                       'roofstyle_Mansard','roofstyle_Shed','roofmatl_Membran','roofmatl_Metal','roofmatl_Roll',
                       'roofmatl_Tar&Grv','roofmatl_WdShake','roofmatl_WdShngl','exterior1st_AsbShng',
                       'exterior1st_AsphShn','exterior1st_BrkComm','exterior1st_BrkFace','exterior1st_CBlock',
                       'exterior1st_CemntBd','exterior1st_HdBoard','exterior1st_ImStucc','exterior1st_MetalSd',
                       'exterior1st_Plywood','exterior1st_PreCast','exterior1st_Stucco','exterior1st_Wd Sdng',
                       'exterior1st_WdShing','exterior2nd_AsbShng','exterior2nd_AsphShn','exterior2nd_Brk Cmn',
                       'exterior2nd_BrkFace','exterior2nd_CBlock','exterior2nd_CmentBd','exterior2nd_HdBoard',
                       'exterior2nd_ImStucc','exterior2nd_MetalSd','exterior2nd_Plywood','exterior2nd_PreCast',
                       'exterior2nd_Stone','exterior2nd_Stucco','exterior2nd_Wd Sdng','exterior2nd_Wd Shng',
                       'masvnrtype_BrkCmn','masvnrtype_BrkFace','masvnrtype_Missing','masvnrtype_Stone',
                       'exterqual_Ex','exterqual_Fa','exterqual_Gd','extercond_Ex','extercond_Fa','extercond_Gd',
                       'extercond_Po','foundation_BrkTil','foundation_PConc','foundation_Slab','foundation_Stone',
                       'foundation_Wood','bsmtqual_Ex','bsmtqual_Fa','bsmtqual_Gd','bsmtqual_No Bsmt','bsmtqual_Po',
                       'bsmtcond_Ex','bsmtcond_Fa','bsmtcond_Gd','bsmtcond_No Bsmt','bsmtcond_Po','bsmtexposure_Av',
                       'bsmtexposure_Gd','bsmtexposure_Mn','bsmtexposure_No Bsmt','bsmtfintype1_ALQ','bsmtfintype1_BLQ',
                       'bsmtfintype1_LwQ','bsmtfintype1_No Bsmt','bsmtfintype1_Rec','bsmtfintype1_Unf',
                       'bsmtfintype2_ALQ','bsmtfintype2_BLQ','bsmtfintype2_GLQ','bsmtfintype2_LwQ',
                       'bsmtfintype2_No Bsmt','bsmtfintype2_Rec','heating_Floor','heating_GasW','heating_Grav',
                       'heating_OthW','heating_Wall','heatingqc_Fa','heatingqc_Gd','heatingqc_Po','heatingqc_TA',
                       'centralair_N','electrical_FuseA','electrical_FuseF','electrical_FuseP','kitchenqual_Ex',
                       'kitchenqual_Fa','kitchenqual_Gd','kitchenqual_Po','functional_Maj1','functional_Maj2',
                       'functional_Min1','functional_Min2','functional_Mod','functional_Sal','fireplacequ_Ex',
                       'fireplacequ_Fa','fireplacequ_Gd','fireplacequ_Po','fireplacequ_TA','garagetype_2Types',
                       'garagetype_Basment','garagetype_BuiltIn','garagetype_CarPort','garagetype_Detchd',
                       'garagetype_No Garage','garagefinish_Fin','garagefinish_No Garage','garagefinish_RFn',
                       'garagequal_Ex','garagequal_Fa','garagequal_Gd','garagequal_No Garage','garagequal_Po',
                       'garagecond_Ex','garagecond_Fa','garagecond_Gd','garagecond_No Garage','garagecond_Po',
                       'paveddrive_N','paveddrive_P','poolqc_Ex','poolqc_Fa','poolqc_Gd','poolqc_TA','fence_GdPrv',
                       'fence_GdWo','fence_MnPrv','fence_MnWw','miscfeature_Gar2','miscfeature_Othr',
                       'miscfeature_Shed','miscfeature_TenC','saletype_COD','saletype_CWD','saletype_Con',
                       'saletype_ConLD','saletype_ConLI','saletype_ConLw','saletype_New','saletype_Oth',
                       'saletype_VWD','salecondition_Abnorml','salecondition_AdjLand','salecondition_Alloca',
                       'salecondition_Family','salecondition_Partial']]

## Initial Feature Selection 


In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso()

In [ ]:
coefs = []
intercepts = []
coefs_dict = {}

alphaRange = np.linspace(1e-3,0.2,20)
for alpha in alphaRange:
          lasso.set_params(alpha=alpha)  
          lasso.fit(num_features, target)
          intercepts.append(lasso.intercept_)
          coefs.append(lasso.coef_)
          coefs_dict[alpha] = lasso.coef_

In [ ]:
for i in range(20):
    alpha = alphaRange[i]
    coef_alpha = coefs_dict[alpha]
    print(i, round(alpha,2), len(coef_alpha[coef_alpha!=0]))

In [ ]:
coef_alpha1 = pd.DataFrame({'feature':full_features.columns, 'coef':coefs_dict[alphaRange[1]]})
picked_feature = coef_alpha1.loc[coef_alpha1.coef != 0,'feature'].tolist()
picked_feature

When lambda = 0.03, 23 out of the total 288 features will be kept non-zero by the lasso model. We will look into these features first for more insights.

## Data Analysis for the 36 picked features

In [ ]:
picked36= ames4[picked_feature]
picked36.head()

In [ ]:
#Check the correlation between all numeric varibales
corr = picked36.iloc[:,:23].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

- It seems 'grlivarea','overallqual','yearbuilt', 'masvnrarea', 'totalbsmtsf', 'garagecars','garagearea', 'house_age', 'reno_age', and 'tbath_abvgr' are correlated with many other features.
- 'garagecars' and 'garagearea' are highl correlated
- 'house_age' is correlated with 'yearbuilt', 'bsmtfullbath' is highly correlated with 'tbath_bsmt', as we bring in one of the feature based on another.